# Advent of Code Problem Nr. 4

> You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine. Maybe it wants to play bingo?

## Setup the data

As always, we'll ingest our data into pandas and import the utilities used for displaying in IPython.

In [20]:
from IPython.display import display, Markdown,Latex
import pandas as pd
import numpy as np

bingo_boards:pd.DataFrame = pd.read_csv('../../inputs/4/bingo-boards.csv',dtype='int',sep='\s+',header=None)
numbers_drawn:pd.Series = pd.read_csv('../../inputs/4/draw-order.csv',dtype='int',header=None).T[0]

With some additional processing, we can get the boards split up so that we can access each bingo-board on its own.

In [21]:
bingo_boards["board_index"]=(bingo_boards.index//5)
bingo_boards["row_index"]=(bingo_boards.index % 5)

bingo_boards = bingo_boards.set_index(["board_index","row_index"])

display(bingo_boards)

0   1   2   3   4
board_index row_index                    
0           0          78  13   8  62  67
            1          42  89  97  16  65
            2           5  12  73  50  56
            3          45  10  63  41  64
            4          49   1  95  71  17
...                    ..  ..  ..  ..  ..
99          0          57  95  40  92  27
            1          65  37  42  90   9
            2          17  72  78  43  45
            3          87  28  48  81  79
            4           7   4  24  67  70

[500 rows x 5 columns]

Now, our data is set up for the future, with a nice pandas multiindex to boot.

-----

## Part One

For the first task, we need to calculate the winning board, which is the one in which the drawn numbers first fill one column or row.

With that board identified, the answer to the puzzle is the sum of all unmarked numbers multiplied by the last number that was just drawn to make the board the winning one.

-----

First, let's write a function that tells us whether a column or row of a board won with a given sequence of numbers.

In [22]:
def winning_sequence_length(board_series: pd.Series,
                            draw_sequence: pd.Series,
                            max_length: int = None) -> 'None|int':
    """
  For a given board_series (a column or row), decide whether it wins the game with the given draw_sequence. If yes, return the shortest prefix of the given draw_sequence that is necessary to win, otherwise return None.
  """
    temp_series = board_series.copy()
    for index, num in draw_sequence.iteritems():
        if max_length and max_length < index: return None
        temp_series = temp_series[temp_series.ne(num)]
        if temp_series.empty: return index + 1


display(Markdown('-----'))
display(Markdown('*Example Usage*:'))
example_row = pd.Series([1, 2, 3])
example_draw = pd.Series([2, 3, 4, 1, 6])
display(
    Markdown(f"""
  The shortest winning sequence for row `{example_row.to_list()}`
  out of draw `{example_draw.to_list()}` is 
  `{winning_sequence_length(example_row, example_draw)}` numbers long.
  """))


-----

*Example Usage*:


  The shortest winning sequence for row `[1, 2, 3]`
  out of draw `[2, 3, 4, 1, 6]` is 
  `4` numbers long.
  

-----
With this, we can apply it to all boards.

In [23]:
board_sequence_lengths = {}


def offer_sequence_length(board_index: int, sequence_length: int):
    if sequence_length and (
            board_index not in board_sequence_lengths
            or sequence_length < board_sequence_lengths[board_index]):
        board_sequence_lengths[board_index] = sequence_length


for board_index, board in bingo_boards.groupby(level=0):
    for _, column in board.iteritems():
        offer_sequence_length(
            board_index,
            winning_sequence_length(
                column,
                numbers_drawn,
                max_length=board_sequence_lengths.get(board_index)))
    for _, row in board.iterrows():
        offer_sequence_length(
            board_index,
            winning_sequence_length(
                row,
                numbers_drawn,
                max_length=board_sequence_lengths.get(board_index)))


As a result, we now have a dictionary that contains each board's winning length, so we can determine which wins first.

In [24]:
first_winning_board_index = min(board_sequence_lengths,
                                key=board_sequence_lengths.get)
winning_board_draw_length = board_sequence_lengths[first_winning_board_index]

display(
    Markdown(
        f'The first board to win is the board with the index `{first_winning_board_index}`, which wins after `{winning_board_draw_length}` numbers have been drawn.'
    ))


The first board to win is the board with the index `94`, which wins after `27` numbers have been drawn.

In [25]:
first_winning_board = bingo_boards.query(f"board_index=={first_winning_board_index}")
display(Markdown(f"*The winning board looks like this:*"))
display(first_winning_board)

*The winning board looks like this:*

0   1   2   3   4
board_index row_index                    
94          0          70  56  80  12  11
            1          35  55  40  71  87
            2          84  27  96  46  85
            3          20  23  26  29  14
            4          58  37  21  75  68

In [26]:
numbers_actually_drawn = numbers_drawn.head(
    winning_board_draw_length).to_list()

not_drawn = [
    val for sublist in first_winning_board.values for val in sublist
    if val not in numbers_actually_drawn
]

result = sum(not_drawn) * numbers_actually_drawn[-1]

display(
    Latex(f"""
    Now we can answer the puzzle:

    \(r = s_{{notDrawn}} \\times drawn_{{last}}={sum(not_drawn)} \\times {numbers_actually_drawn[-1]}={result}\).
    """))


<IPython.core.display.Latex object>

Yay, right again :)

-----

## Part Two

Now we want to *lose* instead of win.
Let's find out which board will win last.

In [27]:
last_winning_board_index = max(board_sequence_lengths,
                               key=board_sequence_lengths.get)
losing_board_draw_length = board_sequence_lengths[last_winning_board_index]

display(
    Markdown(
        f'The last board to win is the board with the index `{last_winning_board_index}`, which wins after `{losing_board_draw_length}` numbers have been drawn.'
    ))


The last board to win is the board with the index `93`, which wins after `85` numbers have been drawn.

In [28]:
last_winning_board = bingo_boards.query(
    f"board_index=={last_winning_board_index}")
display(Markdown(f"*The losing board looks like this:*"))
display(last_winning_board)

*The losing board looks like this:*

0   1   2   3   4
board_index row_index                    
93          0          50  66  43  39  16
            1          88  94  60  70  64
            2          63  80  56  69  36
            3          53  48  32  22  79
            4          59  77  20  30  67

In [29]:
numbers_actually_drawn = numbers_drawn.head(
    losing_board_draw_length).to_list()

not_drawn = [
    val for sublist in last_winning_board.values for val in sublist
    if val not in numbers_actually_drawn
]

result = sum(not_drawn) * numbers_actually_drawn[-1]

display(
    Latex(f"""
    Now we can answer the puzzle:

    \(r = s_{{notDrawn}} \\times drawn_{{last}}={sum(not_drawn)} \\times {numbers_actually_drawn[-1]}={result}\).
    """))


<IPython.core.display.Latex object>

And just like that, we did it!